# Build Docker Image 
In this notebook we will build the docker container that contains the Resnet 152 model, Flask web application, model driver and all dependencies.

In [1]:
import os
from os import path
import json

In [16]:
!rm -r flaskwebapp

In [17]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

In [18]:
!cp resnet_v1_152.ckpt flaskwebapp
!cp synset.txt flaskwebapp
!cp driver.py flaskwebapp
!ls flaskwebapp

driver.py  etc	nginx  resnet_v1_152.ckpt  synset.txt


Below is the module for the Flask web application.

In [19]:
%%writefile flaskwebapp/app.py
from flask import Flask, request
import time
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()


@app.route('/score', methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    predictions = predict_for(request_input)
    return json.dumps({'result': predictions})


@app.route("/")
def healthy():
    return "Healthy"


@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server

Writing flaskwebapp/app.py


In [20]:
%%writefile flaskwebapp/wsgi.py
import sys
sys.path.append('/code/') # FIXME: This is horrible
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


In [21]:
%%writefile flaskwebapp/requirements.txt
pillow
click==6.7
configparser==3.5.0
Flask==0.11.1
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

Writing flaskwebapp/requirements.txt


The configuration for the Nginx. Note that it creates a proxy between ports **80** and **5000**.

In [22]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


The image name below referes to our dockerhub account. If you wish to push the image to your account make sure you change this.

In [2]:
image_name = "masalvar/tfresnet-gpu"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [24]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [25]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()


Writing flaskwebapp/kill_supervisor.py


In [26]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [27]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy pandas scikit-learn && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH

RUN rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install tensorflow-gpu==1.4.1 && \
    pip install -r /code/requirements.txt

EXPOSE 80
CMD ["supervisord", "-c", "/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


In [28]:
!docker build -t $image_name -f $docker_file_location $application_path

Sending build context to Docker daemon 241.6 MB
Step 1/15 : FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
 ---> 547cf50ecba4
Step 2/15 : MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>
 ---> Using cache
 ---> 930029a3d8d8
Step 3/15 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Using cache
 ---> 6a089d765c9d
Step 4/15 : RUN mkdir /code
 ---> Using cache
 ---> bdd9de88a5ea
Step 5/15 : WORKDIR /code
 ---> Using cache
 ---> a7812550d55d
Step 6/15 : ADD . /code/
 ---> Using cache
 ---> 2b812dca554a
Step 7/15 : ADD etc /etc
 ---> Using cache
 ---> e81f3cedc9ca
Step 8/15 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         cmake         curl         git         nginx         supervisor         wget &&         rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 83032a9676aa
Step 9/15 : ENV PYTHON_VERSION 3.5
 

Below we will push the image created to our dockerhub registry. Make sure you have already logged in to the appropriate dockerhub account using the docker login command

In [3]:
!docker push $image_name # If you haven't loged in to the approrpiate dockerhub account you will get an error

The push refers to a repository [docker.io/masalvar/tfresnet-gpu]

2b3068ba: Preparing 
a7e77e46: Preparing 
4c881bb0: Preparing 
a5eef583: Preparing 
6256b4b8: Preparing 
46c4a53a: Preparing 
957ee06a: Preparing 
6006b2f8: Preparing 
27c508b0: Preparing 
e48572eb: Preparing 
748c63e7: Preparing 
b03ecbb7: Preparing 
b2bd3356: Preparing 
4c622b50: Preparing 
ea2bb533: Preparing 
89ea437e: Preparing 
8b9b1b5b: Preparing 
7e77e46: Pushed  1.371 GB/1.342 GBPushing 71.86 MB/132.8 MBlatest: digest: sha256:e76c29bbb1dfa59032ead344e987fe17a03fda3ec6fb998899df312b9c3bb4a8 size: 4091


In [15]:
print('Docker image name {}'.format(image_name)) 

Docker image name masalvar/tfresnet-gpu


### Test locally
Go to the [Test Locally notebook](03_TestLocally.ipynb) to test your Docker image